                                                        Load Bronze Dataset

In [6]:
import pandas as pd

In [9]:
orders= pd.read_csv('C:/Users/asus/bronze/orders.csv')

In [10]:
customers = pd.read_csv('C:/Users/asus/bronze/customers.csv')

In [12]:
print(orders.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8000 entries, 0 to 7999
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   order_id     8000 non-null   object 
 1   customer_id  8000 non-null   object 
 2   amount       8000 non-null   float64
 3   status       8000 non-null   object 
 4   created_at   8000 non-null   object 
dtypes: float64(1), object(4)
memory usage: 312.6+ KB
None


 Cleaning  orders data

In [13]:
print(orders)

     order_id customer_id   amount      status  \
0      A00001      C00850  1442.80     SHIPPED   
1      A00002      C01003  1103.33   CANCELLED   
2      A00003      C00490    70.45  PROCESSING   
3      A00004      C00969  1916.48  PROCESSING   
4      A00005      C00902   660.26     SHIPPED   
...       ...         ...      ...         ...   
7995   A07996      C00013    14.15   DELIVERED   
7996   A07997      C00614  2122.53   DELIVERED   
7997   A07998      C00366  2459.31   DELIVERED   
7998   A07999      C00892  1648.91    RETURNED   
7999   A08000      C00098  1918.36     SHIPPED   

                            created_at  
0     2025-12-22T14:27:19.903124+00:00  
1     2026-02-11T14:41:30.903124+00:00  
2     2026-01-14T02:52:25.903124+00:00  
3     2026-01-27T01:47:19.903124+00:00  
4     2025-11-29T06:50:01.903124+00:00  
...                                ...  
7995  2025-11-21T07:34:01.903124+00:00  
7996  2025-12-08T15:20:19.903124+00:00  
7997  2026-01-13T04:32:55.9031

 Clean orders Table

standardize status

In [15]:
orders['status'] = orders['status'].str.strip().str.lower()

convert created_at datetime

In [17]:
orders['created_at'] = pd.to_datetime(orders['created_at'],errors ='coerce')

Filling missing amount with 0

In [18]:
orders['amount'] = orders['amount'].fillna(0)

Droping rows without customers_id

In [21]:
orders = orders.dropna(subset =['customer_id'])

In [25]:
orders.isna().sum()

order_id       0
customer_id    0
amount         0
status         0
created_at     0
dtype: int64

In [26]:
orders.head()

,order_id,customer_id,amount,status,created_at
0,A00001,C00850,1442.80,shipped,2025-12-22 14:27:19.903124+00:00
1,A00002,C01003,1103.33,cancelled,2026-02-11 14:41:30.903124+00:00
2,A00003,C00490,70.45,processing,2026-01-14 02:52:25.903124+00:00
3,A00004,C00969,1916.48,processing,2026-01-27 01:47:19.903124+00:00
4,A00005,C00902,660.26,shipped,2025-11-29 06:50:01.903124+00:00


Cleaning Customers Table

In [28]:
print(customers)

     customer_id       city
0         C00001   Nürnberg
1         C00002    Hamburg
2         C00003     Berlin
3         C00004     Bremen
4         C00005  Frankfurt
...          ...        ...
1195      C01196    Dresden
1196      C01197     Bremen
1197      C01198    München
1198      C01199    Hamburg
1199      C01200  Frankfurt

[1200 rows x 2 columns]


In [29]:
                    # Cleaning Cities name with title func
customers['city'] = customers['city'].str.strip().str.title()

Saving the file in folder as parquet

In [51]:
orders.to_parquet(
    r"C:\Users\asus\Downloads\Project.de\Silver\silver_orders.parquet",
    index=False
)

In [52]:
customers.to_parquet(
    r"C:\Users\asus\Downloads\Project.de\Silver\silver_customers.parquet",
    index=False
)

Explode Items of JSON into rows

In [35]:
import json

In [48]:
rows = []
with open("bronze/order_items.jsonl") as f:
    for line in f:
        rec =json.loads(line)
        for item in rec['items']:
            rows.append({
                "order_id": rec["order_id"],
                "sku": item["sku"],
                "qty": item["qty"],
                "price": item["price"]

            })
items_df = pd.DataFrame(rows)

In [49]:
items_df.head()

,order_id,sku,qty,price
0,A00001,S1,3,147.95
1,A00001,S10,5,199.79
2,A00002,S9,4,75.57
3,A00002,S9,5,81.67
4,A00002,S6,2,196.35


Saving file in Silver Folder as parquett

In [53]:
items_df.to_parquet(
    r"C:\Users\asus\Downloads\Project.de\Silver\silver_items.parquett",
    index=False
)